<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

In [1]:
# Parameters
RAW_DIR = "data/raw/"
PROCESSED_DIR = "data/processed/"
RANDOM_STATE = 42


In [2]:
# Parameters (papermill)
from pathlib import Path

RAW_DIR = "data/raw"          # sobreescrito por papermill si usas --parameters
PROCESSED_DIR = "data/processed"
RANDOM_STATE = 42

RAW_DIR = Path(RAW_DIR)
PROCESSED_DIR = Path(PROCESSED_DIR)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# LIBRERIAS 
import pandas as pd
import numpy as np
import hashlib
import re
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import joblib
import os

In [4]:
# rocesamiento del archivo ML_event_windows_estaciones.csv
# Rutas de entrada y salida
input_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/ML_event_windows_estaciones.csv")
output_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_windows_estaciones.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Cargar los datos 
df = pd.read_csv(input_path)

# === Selección y renombramiento de columnas ===
df_proc = df.rename(columns={
    "@timestamp": "timestamp",
    "related.user": "src_user",
    "winlog.event_data.ObjectType": "object_type",
    "winlog.event_data.ObjectName": "object_name",
    "event_critical.description": "description",
    "event_critical.event_id": "event_id",
    "log.level": "severity",
    "event.action": "action"
})

# Agregar columnas faltantes para el esquema estándar
df_proc["src_ip"] = None
df_proc["dst_ip"] = None
df_proc["protocol"] = "N/A"
df_proc["log_source"] = "windows_estaciones"

# Renombrar columnas y seleccionar las importantes 
df_norm = pd.DataFrame()
df_norm["timestamp"] = df["@timestamp"]
df_norm["src_user"] = df["related.user"]
df_norm["action"] = df["event.action"]
df_norm["description"] = df["event_critical.description"]
df_norm["object_type"] = df["winlog.event_data.ObjectType"]
df_norm["object_name"] = df["winlog.event_data.ObjectName"]
df_norm["channel"] = df["winlog.channel"]
df_norm["log_level"] = df["log.level"]
df_norm["log_source"] = df["agent.name"]

# Normalizar valores de log_level a formato uniforme 
df_norm["log_level"] = df_norm["log_level"].str.lower().replace({
    "información": "info",
    "advertencia": "warning",
    "error": "error"
})

# Limpiar espacios en blanco y valores vacíos
df_norm = df_norm.fillna("-")

# Reordenar columnas al esquema estándar + útiles adicionales
ordered_cols = [
    "timestamp", "src_ip", "dst_ip", "protocol", "action", "description",
    "event_id", "severity", "src_user", "object_type", "object_name", "log_source"
]

df_proc = df_proc[ordered_cols]

# Guardar archivo procesado
df_norm.to_csv(output_path, index=False)
print(f" Archivo procesado guardado en: {output_path}")

 Archivo procesado guardado en: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_windows_estaciones.csv


In [5]:
# Procesamiento del archivo ML_WAF.csv
# Rutas
input_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/ML_WAF.csv")
output_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_waf.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_csv(input_path)

# Renombrar y mapear columnas
df_proc = df.rename(columns={
    "@timestamp": "timestamp",
    "source.ip": "src_ip",
    "destination.domain": "dst_ip",
    "network.protocol": "protocol",
    "http.request.method": "action",
    "event.id": "event_id",
    "rule.category": "category",
    "url.original": "url"
})

# Crear columna 'description' uniendo categoría de regla + URL
df_proc["description"] = df_proc["category"].fillna('') + " - " + df_proc["url"].fillna('')

# Agregar columnas faltantes
df_proc["severity"] = None
df_proc["src_user"] = None
df_proc["object_type"] = None
df_proc["object_name"] = None
df_proc["log_source"] = "waf"

# Reordenar columnas
ordered_cols = [
    "timestamp", "src_ip", "dst_ip", "protocol", "action", "description",
    "event_id", "severity", "src_user", "object_type", "object_name", "log_source"
]
df_proc = df_proc[ordered_cols]

# Guardar el archivo procesado
df_proc.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f" Archivo procesado: {output_path}")

 Archivo procesado: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_waf.csv


In [6]:
# Procesamiento del archivo ML_antimalware.csv
# Rutas
input_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/ML_antimalware.csv")
output_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_antimalware.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_csv(input_path)

# Renombrar columnas relevantes
df_proc = df.rename(columns={
    "@timestamp": "timestamp",
    "network.protocol": "protocol",
    "syslog.type": "action",
    "file.name": "file_name",
    "file.extension": "file_ext",
    "risk.calculated_level": "risk_level",
    "file.hash.sha256": "event_id"
})

# Construir descripción combinada
df_proc["description"] = (
    "Archivo: " + df_proc["file_name"].fillna('') +
    "." + df_proc["file_ext"].fillna('') +
    " | Riesgo: " + df_proc["risk_level"].astype(str)
)

# Agregar columnas faltantes
df_proc["src_ip"] = None
df_proc["dst_ip"] = None
df_proc["severity"] = df_proc["risk_level"]
df_proc["src_user"] = None
df_proc["object_type"] = "file"
df_proc["object_name"] = df_proc["file_name"]
df_proc["log_source"] = "antimalware"

# Reordenar columnas
ordered_cols = [
    "timestamp", "src_ip", "dst_ip", "protocol", "action", "description",
    "event_id", "severity", "src_user", "object_type", "object_name", "log_source"
]
df_proc = df_proc[ordered_cols]

# Guardar archivo procesado
df_proc.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f" Archivo procesado: {output_path}")

 Archivo procesado: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_antimalware.csv


In [7]:
# Procesamiento del archivo ML_antivirus_kaspersky.csv
# Rutas
input_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/ML_antivirus_kaspersky.csv")
output_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_kaspersky.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_csv(input_path)

# Renombrar columnas relevantes
df_proc = df.rename(columns={
    "@timestamp": "timestamp",
    "host.ip": "src_ip",
    "event.reason": "action",
    "file.path": "object_name",
    "file.hash.sha256": "event_id",
    "virus.name": "virus_name",
    "danger.type": "danger_type",
    "rule.description": "rule_description"
})

# Crear descripción combinada
df_proc["description"] = (
    "Virus: " + df_proc["virus_name"].fillna('') +
    " | Tipo: " + df_proc["danger_type"].fillna('') +
    " | " + df_proc["rule_description"].fillna('')
)

# Agregar columnas faltantes
df_proc["dst_ip"] = None
df_proc["protocol"] = "N/A"
df_proc["severity"] = df_proc["danger_type"]
df_proc["src_user"] = df["host.hostname"]
df_proc["object_type"] = "file"
df_proc["log_source"] = "antivirus_kaspersky"

# Reordenar columnas
ordered_cols = [
    "timestamp", "src_ip", "dst_ip", "protocol", "action", "description",
    "event_id", "severity", "src_user", "object_type", "object_name", "log_source"
]
df_proc = df_proc[ordered_cols]

# Guardar archivo procesado
df_proc.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f" Archivo procesado: {output_path}")


 Archivo procesado: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_kaspersky.csv


In [8]:
# Procesamiento del archivo Logs-antivirus_etiquetado.csv
# Rutas
input_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/Logs-antivirus_etiquetado.csv")
output_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_antivirus_etiquetado.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_csv(input_path)

# Renombrar columnas relevantes
df_proc = df.rename(columns={
    "@timestamp": "timestamp",
    "host.ip": "src_ip",
    "file.path": "object_name",
    "file.hash.sha256": "event_id",
    "virus.name": "virus_name",
    "danger.type": "danger_type",
    "rule.description": "rule_description"
})

# Crear columna description
df_proc["description"] = (
    "Virus: " + df_proc["virus_name"].fillna('') +
    " | Tipo: " + df_proc["danger_type"].fillna('') +
    " | " + df_proc["rule_description"].fillna('')
)

# Agregar campos faltantes
df_proc["dst_ip"] = None
df_proc["protocol"] = "N/A"
df_proc["severity"] = df_proc["danger_type"]
df_proc["action"] = "Detectado"
df_proc["src_user"] = df["host.hostname"]
df_proc["object_type"] = "file"
df_proc["log_source"] = "antivirus_etiquetado"

# Reordenar
ordered_cols = [
    "timestamp", "src_ip", "dst_ip", "protocol", "action", "description",
    "event_id", "severity", "src_user", "object_type", "object_name", "log_source"
]
df_proc = df_proc[ordered_cols]

# Guardar archivo final
df_proc.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f" Archivo procesado: {output_path}")


 Archivo procesado: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_antivirus_etiquetado.csv


In [9]:
# Procesamient odel archivo ML_event_windows_server_correo_interno.csv
# Rutas
input_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/raw/ML_event_windows_server_correo_interno.csv")
output_path = Path("/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_windows_server_correo.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_csv(input_path)

# Renombrar columnas clave
df_proc = df.rename(columns={
    "@timestamp": "timestamp",
    "related.user": "src_user",
    "winlog.event_data.ObjectType": "object_type",
    "winlog.event_data.ObjectName": "object_name",
    "event_critical.description": "description",
    "event_critical.event_id": "event_id",
    "log.level": "severity",
    "event.action": "action",
    "winlog.logon.failure.reason": "failure_reason"
})

# Unir motivo de fallo con la descripción
df_proc["description"] = df_proc["description"].fillna('') + " | " + df_proc["failure_reason"].fillna('')

# Agregar columnas faltantes
df_proc["src_ip"] = None
df_proc["dst_ip"] = None
df_proc["protocol"] = "N/A"
df_proc["log_source"] = "windows_server_correo"

# Reordenar columnas
ordered_cols = [
    "timestamp", "src_ip", "dst_ip", "protocol", "action", "description",
    "event_id", "severity", "src_user", "object_type", "object_name", "log_source"
]
df_proc = df_proc[ordered_cols]

# Guardar archivo final
df_proc.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f" Archivo procesado: {output_path}")



 Archivo procesado: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_windows_server_correo.csv


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
from pathlib import Path
# Rutas
BASE_PATH = Path("/Users/alonso/Downloads/Tesis_SIEM_ML")
PROCESSED_LOGS_PATH = BASE_PATH / "data" / "processed_logs"

ARCHIVOS = {
    "windows": "logs_windows_estaciones.csv",
    "kaspersky": "logs_kaspersky.csv",
    "waf": "logs_waf.csv",
    "antimalware": "logs_antimalware.csv"
}

# Columnas estándar
columnas_requeridas = [
    'timestamp', 'src_ip', 'dst_ip', 'protocol', 'action',
    'description', 'event_id', 'severity', 'src_user',
    'object_type', 'object_name', 'log_source'
]

# Función para procesar cada archivo
def cargar_log_estandar(nombre_archivo, fuente):
    ruta = PROCESSED_LOGS_PATH / nombre_archivo
    if not ruta.exists():
        print(f" Archivo no encontrado: {nombre_archivo}")
        return pd.DataFrame()  # Devuelve DataFrame vacío
    
    df = pd.read_csv(ruta)

    # Asegurar columnas requeridas
    for col in columnas_requeridas:
        if col not in df.columns:
            df[col] = None

    df = df[columnas_requeridas]
    df["fuente"] = fuente
    return df

# Unificación
df_final = pd.DataFrame()

for fuente, archivo in ARCHIVOS.items():
    df_temp = cargar_log_estandar(archivo, fuente)
    df_final = pd.concat([df_final, df_temp], ignore_index=True)

# Limpieza básica
df_final.dropna(subset=["timestamp", "action"], inplace=True)
df_final.reset_index(drop=True, inplace=True)

# Guardado final
output_path = PROCESSED_LOGS_PATH / "logs_normalizados.csv"
df_final.to_csv(output_path, index=False)
print(f" Archivo unificado guardado en: {output_path}")

OSError: [Errno 5] Input/output error